# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [ ]:
# imports

import os, json, gradio as gr, anthropic, google.generativeai
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# Other LLMs
DALL_E_MODEL = "dall-e-3"

CLAUDE_MODEL = "claude-sonnet-4-20250514"
claude = anthropic.Anthropic()

google_api_key = os.getenv('GOOGLE_API_KEY')
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")
    
GEMINI_MODEL= "gemini-2.5-flash"
gemini = google.generativeai.configure()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# Just take in history
def chat(history):
    message = history[-1]["content"] # Get the last message from the user
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response_tool, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response_tool)
        image = artist(city)
        print("Avail image for: ", city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)


    # After getting the final response from OpenAI
    final_response_content = response.choices[0].message.content
    history.append({"role": "assistant", "content": final_response_content})

    # The return value should be a tuple of (history, image)
    return history, image

# gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
# get_ticket_price("London")

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]
# print(tools)

In [ ]:
# Simulate the booking process by simply returning a confirmation string.

def book_flight(destination_city, number_of_passengers, booking_date):
    """
    Simulates booking a flight.
    """
    print(f"Tool book_flight called for {destination_city} for {number_of_passengers} passengers on {booking_date}")
    return f"Your booking to {destination_city} for {number_of_passengers} passengers on {booking_date} has been confirmed. Your booking reference is BKG-{hash(destination_city + str(number_of_passengers) + str(booking_date))}"

In [ ]:
# Tool definition for book_flight

booking_function = {
    "name": "book_flight",
    "description": "Books a flight for a customer. Call this whenever a customer asks to book a flight.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city the customer wants to fly to."
            },
            "number_of_passengers": {
                "type": "integer",
                "description": "The number of passengers for the booking."
            },
            "booking_date": {
                "type": "string",
                "description": "The date of the flight booking in YYYY-MM-DD format."
            }
        },
        "required": ["destination_city", "number_of_passengers", "booking_date"],
        "additionalProperties": False
    }
}

In [ ]:
# Add the new booking_function to the existing tools list.

tools.append({"type": "function", "function": booking_function})

In [ ]:
# To translate to a given language

def translate_text(text, target_language):
    """
    Translates text to a specified language.
    
    Args:
        text (str): The text to translate.
        target_language (str): The language to translate the text into.
        
    Returns:
        str: The translated text or an error message.
    """
    print(f"Tool translate_text called to translate to {target_language}")
    
    # Use a system prompt to instruct the model to perform a translation
    system_prompt_for_language = f"You are a helpful translation assistant. Translate the following text into {target_language}. Only provide the translated text without any additional conversational text."
    
    try:
        # # Using OpenAI
        # response = openai.chat.completions.create(
        #     model=MODEL,
        #     messages=[
        #         {"role": "system", "content": system_prompt_for_language},
        #         {"role": "user", "content": text}
        #     ],
        # )
        # result = response.choices[0].message.content
        # return result

        
        # # Using Gemini
        # gemini = google.generativeai.GenerativeModel(
        #     model_name=GEMINI_MODEL,
        #     system_instruction=system_prompt_for_language
        # )
        # response = gemini.generate_content(text)
        # result = response.text
        # return result

        
        # Using Claude
        response = claude.messages.create(
            model=CLAUDE_MODEL,
            max_tokens=200,
            temperature=0.7,
            system=system_prompt_for_language,
            messages=[
                {"role": "user", "content": text},
            ],
        )
        result = response.content[0].text
        return result
        
    except Exception as e:
        print(f"Error during translation: {e}")
        return "Sorry, I encountered an error and could not complete the translation."

In [ ]:
# Tool definition for translate_text

translation_function = {
    "name": "translate_text",
    "description": "Translates a given text to a specified target language. Call this whenever a customer asks for a translation.",
    "parameters": {
        "type": "object",
        "properties": {
            "text": {
                "type": "string",
                "description": "The text to be translated."
            },
            "target_language": {
                "type": "string",
                "description": "The language to translate the text into (e.g., 'French', 'Spanish', 'Swahili')."
            }
        },
        "required": ["text", "target_language"],
        "additionalProperties": False
    }
}

In [ ]:
# Integrate the tool

tools.append({"type": "function", "function": translation_function})

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

# Handle multiple tools
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    destination_city = None
    translated_text = None

    if function_name == "get_ticket_price":
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response_content = json.dumps({"destination_city": city, "price": price})
        destination_city = city
    elif function_name == "book_flight":
        destination_city = arguments.get('destination_city')
        number_of_passengers = arguments.get('number_of_passengers')
        booking_date = arguments.get('booking_date')
        confirmation = book_flight(destination_city, number_of_passengers, booking_date)
        response_content = json.dumps({"confirmation_message": confirmation})
    elif function_name == "translate_text":
        text = arguments.get('text')
        target_language = arguments.get('target_language')
        translated_text = translate_text(text, target_language)
        response_content = json.dumps({"translated_text": translated_text})
    else:
        response_content = json.dumps({"error": f"Unknown tool: {function_name}"})

    response = {
        "role": "tool",
        "content": response_content,
        "tool_call_id": tool_call.id
    }
    return response, destination_city

In [ ]:
# gr.ChatInterface(fn=chat, type="messages").launch()

# Let's go multi-modal!!

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

Let's put this in a function called artist.

### Price alert: each time I generate an image it costs about 4 cents - don't go crazy with images!

In [ ]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model=DALL_E_MODEL,
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
# image = artist("New York City")
# display(image)

## Audio (NOTE - Audio is optional for this course - feel free to skip Audio if it causes trouble!)

And let's make a function talker that uses OpenAI's speech model to generate Audio

### Troubleshooting Audio issues

If you have any problems running this code below (like a FileNotFound error, or a warning of a missing package), you may need to install FFmpeg, a very popular audio utility.

**For Mac Users**

1. Install homebrew if you don't have it already by running this in a Terminal window and following any instructions:  
`/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"`

2. Then install FFmpeg with `brew install ffmpeg`

3. Verify your installation with `ffmpeg -version` and if everything is good, within Jupyter Lab do Kernel -> Restart kernel to pick up the changes

Message me or email me at ed@edwarddonner.com with any problems!

## To check you now have ffmpeg and can access it here

Excecute the next cell to see if you get a version number. (Putting an exclamation mark before something in Jupyter Lab tells it to run it as a terminal command rather than python code).

If this doesn't work, you may need to actually save and close down your Jupyter lab, and start it again from a new Terminal window (Mac) or Anaconda prompt (PC), remembering to activate the llms environment. This ensures you pick up ffmpeg.

And if that doesn't work, please contact me!

In [ ]:
!ffmpeg -version
!ffprobe -version
!ffplay -version

# For Mac users - and possibly many PC users too

This version should work fine for you. It might work for Windows users too, but you might get a Permissions error writing to a temp file. If so, see the next section!

As always, if you have problems, please contact me! (You could also comment out the audio talker() in the later code if you're less interested in audio generation)

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
# talker("Well, hi there")

In [ ]:
# To transcribe an audio prompt/input

import tempfile
from pydub import AudioSegment
from pydub.playback import play

def transcribe_audio(audio_file):
    """
    Transcribes an audio file using OpenAI's Whisper model.
    """
    if audio_file is None:
        return ""
    
    # The Gradio Audio component returns a tuple (sample_rate, numpy_array)
    # We need to save this to a file to pass to the OpenAI API
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmpfile:
        audio = AudioSegment.from_file(audio_file, format="wav")
        audio.export(tmpfile.name, format="wav")
        
        with open(tmpfile.name, "rb") as audio_file_obj:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file_obj
            )
        return transcript.text

In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image = gr.Image(height=500)
    with gr.Row():
        # entry = gr.Textbox(label="Chat with our AI Assistant:")
        entry = gr.Textbox(label="Chat with our AI Assistant:", scale=4)
        submit_btn = gr.Button("Submit", scale=1)
    with gr.Row():
        # Provide a microphone input
        audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Speak to our AI Assistant", scale=4)
        submit_audio_btn = gr.Button("Submit Audio", scale=1)


    with gr.Row():
        languages = ["English", "Swahili", "French", "Chinese", "German"]
        language_dropdown = gr.Dropdown(
            label="Select a language for translation",
            choices=languages,
            value=languages[0]  # Default to English
        )

        audio_options = ["Yes", "No"]
        audio_dropdown = gr.Dropdown(
            label="Select whether to respond with audio",
            choices=audio_options,
            value=audio_options[1]  # Default to No
        )
        
    with gr.Row():
        clear = gr.Button("Clear")

    def user_message_updater(user_message, history):
        return "", history + [[user_message, None]]

    def chat_with_assistant(history, target_language, use_audio_output):
        message = history[-1][0] # Get the user's message from the last list in history
    
        messages = [{"role": "system", "content": system_message}]
        for msg_user, msg_assistant in history:
            messages.append({"role": "user", "content": msg_user})
            if msg_assistant:
                messages.append({"role": "assistant", "content": msg_assistant})
    
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

        image = None
        
        if response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            response_tool, city = handle_tool_call(message)

            # Check if a city was returned from the tool call to generate an image
            if city:
                image = artist(city) # Generate an image to represent the target City

            messages.append(message.model_dump()) # Append message as a dictionary using .model_dump()
            messages.append(response_tool)
            
            response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    
        final_response_content = response.choices[0].message.content
        history[-1][1] = final_response_content # Update the last message with the assistant's reply

        if target_language != "English": # Assuming "English" is the default and no translation is needed
            translated_response = translate_text(final_response_content, target_language)
            final_response_content = translated_response

        history[-1][1] = final_response_content

        if use_audio_output != "No":
            talker(final_response_content)

        return history, image # Return a tuple of (the updated history, an image)

    # This function ties together the transcription and the chat logic
    def transcribe_and_chat(audio_file, history, target_language, use_audio_output):
        if audio_file:
            # Transcribe the audio file to text
            transcribed_text = transcribe_audio(audio_file)
            
            # Update history with the transcribed text
            new_history = history + [[transcribed_text, None]]
            
            # Call the main chat function with the new history
            return chat_with_assistant(new_history, target_language, use_audio_output)
        else:
            return history, None

    # The event listeners are updated to be triggered by both the textbox and the new button
    entry.submit(
        user_message_updater,
        inputs=[entry, chatbot],
        outputs=[entry, chatbot],
        queue=False
    ).then(
        chat_with_assistant, 
        inputs=[chatbot, language_dropdown, audio_dropdown],
        outputs=[chatbot, image]
    )

    submit_btn.click(
        user_message_updater,
        inputs=[entry, chatbot],
        outputs=[entry, chatbot],
        queue=False
    ).then(
        chat_with_assistant,
        inputs=[chatbot, language_dropdown, audio_dropdown],
        outputs=[chatbot, image]
    )

    # Event listener to trigger on audio stop
    audio_input.stop(
        transcribe_and_chat,
        inputs=[audio_input, chatbot, language_dropdown, audio_dropdown],
        outputs=[chatbot, image],
        queue=False
    )

    submit_audio_btn.click(
        transcribe_and_chat,
        inputs=[audio_input, chatbot, language_dropdown, audio_dropdown],
        outputs=[chatbot, image],
        queue=False
    )
    
    clear.click(lambda: None, inputs=None, outputs=[chatbot, image], queue=False)

ui.launch(inbrowser=True)